# Ch.8 영상 매칭과 추적
## 8.1 비슷한 그림 찾기
### 8.1.1 평균 해시 매칭
- 픽셀 전체의 평균 값을 구해 평균보다 크면 1, 작으면 0으로 변환

### 유클리드 거리 & 해밍 거리
- 평균 해시 매칭을 통해 구한 영상들로 비슷한 정도를 측정하기 위한 방법
*** 유클리드 거리 : 유클리드 거리는 두값의 차리로 거리를 계산 ***
*** 해밍 거리 : 해밍 거리는 두값의 길이가 같을 때, 같은 자리에서의 값이 서로 다른 것이 몇 개인지를 비교 ***
- 영상에서는 해밍 거리가 더 적합

In [28]:
# tar.gz 파일 압축해제
# cmd에서 파일의 directory로 이동 -> tar -zxvf [압축파일명] -C [new folder]

import cv2
import numpy as np
import glob

# 영상 읽기 및 표시
img = cv2.imread('img/pistol.jpg')
cv2.imshow('query', img)

# 비교할 영상들이 있는 경로 
search_dir = 'C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories'

# 이미지를 16x16 크기의 평균 해쉬로 변환 
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (16, 16))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

# 해밍거리 측정 함수
def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

# 권총 영상의 해쉬 구하기
query_hash = img2hash(img)

# 이미지 데이타 셋 디렉토리의 모든 영상 파일 경로
img_path = glob.glob(search_dir+'/**/*.jpg')
for path in img_path:
    # 데이타 셋 영상 한개 읽어서 표시
    img = cv2.imread(path)
    cv2.imshow('searching...', img)
    cv2.waitKey(5)
    # 데이타 셋 영상 한개의 해시 
    a_hash = img2hash(img)
    # 해밍 거리 산출 ---⑧
    dst = hamming_distance(query_hash, a_hash)
    if dst/256 < 0.25: # 해밍거리 25% 이내만 출력
        print(path, dst/256)
        cv2.imshow(path, img)
cv2.destroyWindow('searching...')
cv2.waitKey(0)
cv2.destroyAllWindows()

C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\BACKGROUND_Google\image_0398.jpg 0.234375
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\binocular\image_0011.jpg 0.23828125
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\Faces_easy\image_0419.jpg 0.2421875
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0001.jpg 0.2421875
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0015.jpg 0.24609375
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0017.jpg 0.23828125
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0018.jpg 0.1953125
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0019.jpg 0.23828125
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0021.jpg 0.171875
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategories\revolver\image_0022.jpg 0.21484375
C:/Users/ChangHo Kim/Downloads/folder/101_ObjectCategor

### 8.1.2 템플릿 매칭
- result = cv2.matchTemplate(img, template, method[, result, mask])
    - template : 템플릿 영상
    - method : 매칭 메서드
        - cv2.TM_SQDIFF : 제곱 차이 매칭, 완벽 매칭: 0, 나쁜 매칭: 큰 값
        - cv2.TM_SQDIFF_NORMED : 제곱 차이 매칭의 정규화
        - cv2.TM_CCORR : 상관관계 매칭, 완벽 매칭: 큰 값, 나쁜 매칭: 0
        - cv2.TM_CCORR_NORMED : 상관관계 매칭의 정규화
        - cv2.TM_CCOEFF : 상관계수 매칭, 완벽 매칭: 1, 나쁜 매칭: -1
        - cv2.TM_CCOEFF_NORMED : 상관계수 매칭의 정규화
    - result : 매칭 결과, (W-w+1)x(H-h+1) 크기의 2차원 배열
        - W, H : img의 열과 행
        - w, h : template의 열과 행
    - mask : TM_SQDIFF, TM_CCORR_NORMED인 경우 사용할 마스크

- minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(src[, mask])
    - src : 입력 1채널 배열
    - minVal, maxVal : 배열 전체에서 최소, 최대 값
    - minLoc, maxLoc : 최소 값과 최대 값의 좌표 (x, y)

In [2]:
import cv2
import numpy as np

img = cv2.imread('img/figures.jpg')
template = cv2.imread('img/taekwonv1.jpg')
th, tw = template.shape[:2]
cv2.imshow('template', template)

# 세가지 매칭 메서드 순회
methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']

for i, method_name in enumerate(methods):
    img_draw = img.copy()
    method = eval(method_name)
    # 템플릿 매칭
    res = cv2.matchTemplate(img, template, method)
    # 최대, 최소 값과 그 좌표 구하기
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    print(method_name, min_val, max_val, min_loc, max_loc)

    # TM_SQDIFF_NORMED의 경우 최소 값이 좋은 매칭, 나머지는 그 반대
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        match_val = min_val

    else:
        top_left = max_loc
        match_val = max_val
    
    # 매칭 좌표를 구해서 사각형 표시
    bottom_right = (top_left[0] + tw, top_left[1] + th)
    cv2.rectangle(img_draw, top_left, bottom_right, (0, 0, 255), 2)

    # 매칭 포인트 표시
    cv2.putText(img_draw, str(match_val), top_left, cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.imshow(method_name, img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.TM_CCOEFF_NORMED -0.1780252307653427 0.5131933093070984 (42, 0) (208, 43)
cv2.TM_CCORR_NORMED 0.827332615852356 0.9238022565841675 (85, 6) (208, 43)
cv2.TM_SQDIFF_NORMED 0.17028295993804932 0.36860838532447815 (208, 43) (86, 7)


## 8.2 영상의 특징과 키 포인트
### 8.2.1 코너 특징 검출
- corner : 엣지와 엣지가 만나는 곳
- dst = cv2.cornerHarris(src, blockSize, ksize, k[, dst, borderType])
    - src : 그레이 스케일
    - blockSize : 이웃 픽셀 범위
    - ksize : 코너 검출 상수 (경험적 상수: 0.04~0.06)
    - dst : 검출 결과 (src와 같은 크기의 1채널 배열, 변화량의 값, 지역 최대 값이 코너점을 의미)
    - borderType : 외곽 영역 보정 형식

In [4]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 해리스 코너 검출
corner = cv2.cornerHarris(gray, 2, 3, 0.04)
# 변화량 결과의 최대값 10% 이상의 좌표 구하기
coord = np.where(corner > 0.1*corner.max())
coord = np.stack((coord[1], coord[0]), axis=-1)

# 코너 좌표에 동그라미 그리기
for x, y in coord:
    cv2.circle(img, (x, y), 5, (0, 0, 255), 1, cv2.LINE_AA)

# 변화량을 영상으로 표현하기 위해서 0~255로 정규화
corner_norm = cv2.normalize(corner, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
# 화면에 출력
corner_norm = cv2.cvtColor(corner_norm, cv2.COLOR_GRAY2BGR)
merged = np.hstack((corner_norm, img))
cv2.imshow('Harris Corner', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

- 시와 토마시가 개선한 알고리즘
- corners = cv2.goodFeaturesToTrack(img, maxCorners, qualityLevel, minDistance[, corners, mask, blockSize, useHarrisDetector, k])
    - maxCorners : 얻고 싶은 코너 개수, 강한 것 순
    - qualityLevel : 코너로 판단할 스레시홀드 값
    - minDistance : 코너 간 최소 거리
    - blockSize = 3 : 코너 주변 영역의 크기
    - useHarrisDetector=False : 코너 검출 방법 선택
        - True : 해리스 코너 검출 방법 / False : 시와 토마시 검출 방법
    - k : 해리스 코너 검출 방법에 사용할 k 개수
    - corners : 코너 검출 좌표 결과, (Nx1x2) 크기의 배열, 실수 값이므로 정수로 변형 필요

In [14]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 시-토마스 코너 검출
corners = cv2.goodFeaturesToTrack(gray, 80, 0.01, 20)
# 실수 좌표를 점수 좌표로 변환
corners = np.int32(corners)

# 코너 좌표에 동그라미 그리기
for corner in corners:
    x, y = corner[0]
    cv2.circle(img, (x, y), 5, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('Corners', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 8.2.2 키 포인트와 특징 검출기
- keypoints = detector.detect(img, [, mask]) : 키포인트 검출 함수
    - img : 입력 (바이너리 스케일)
    - mask : 검출 제외 마스크
    - keypoints : 특징점 검출 결과 (list)
<br>
- KeyPoint : 특징점 정보를 담는 객체
    - pt : 키포인트(x, y) 좌표, float 타입으로 정수로 변환 필요
    - size : 의미 있는 키 포인트 이웃의 반지름
    - angle : 특징점 방향 (시계 방향, -1=의미 없음)
    - response : 특징점 반응 강도
    - octave : 발견된 이미지 피라미드 계층
    - class_id : 키포인트가 속한 객체 ID
<br>
- 키포인트를 영상에 표시해주는 전용 함수
- outImg = cv2.drawKeypoints(img, keypoints, outImg[, color[, flags]])
    - keypoints : 표시할 키 포인트 리스트
    - outImg : result
    - color : 표시할 색상 (default: random)
    - flags : 표시 방법 선택 플래그
        - cv2.DRAW_MATCHES_FLAGS_DEFAULT : 좌표 중심에 동그라미만 그림 (default)
        - cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS : 동그라미의 크리를 size와 angle 반영해서 그림
<br>
<br>
### 8.2.3 GFTTDetector
- detector = cv2.GFTTDetector_create([, maxCorners[, qualityLevel, minDistance, blockSize, useHarrisDetector, k]])
    - 인자의 모든 내용은 cv2.goodFeaturesToTrack()과 동일

In [15]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# gftt (good feature to trac)
gftt = cv2.GFTTDetector_create()
# key pts
keypoints = gftt.detect(gray, None)
# key pts 그리기
img_draw = cv2.drawKeypoints(img, keypoints, None)

cv2.imshow('GFTT detector', img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 8.2.4 FAST (feature from accelerated segment test)
- detector = cv2.FastFeatureDetector_create([threshold[, nomaxSuppresion, type]])
    - threshold = 10 : 코너 판단 임계값
    - nomaxSuppresion = True : 최대 점수가 아닌 코너 억제
    - type : edge 검출 패턴
        - cv2.FastFeatureDetector_TYPE_9_16 : 16개 중 9개 연속 (기본값)
        - cv2.FastFeatureDetector_TYPE_7_12 : 12개 중 7개 연속
        - cv2.FastFeatureDetector_TYPE_5_8 : 8개 중 5개 연속

In [20]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# gftt (good feature to trac)
fast = cv2.FastFeatureDetector_create(100)
# key pts
keypoints = fast.detect(gray, None)
# key pts 그리기
img = cv2.drawKeypoints(img, keypoints, None)

cv2.imshow('GFTT detector', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 8.2.5 SimpleBlobDetector
- Binaray Large Object (BLOB)는 바이너리 스케일 이미지의 연결된 픽셀 그룹을 의미함
- 자잘한 객체는 노이즈로 판단하고 특정 크기 이상의 큰 객체에만 관심을 두는 방법
- detector = cv2.SimpleBlobDetector_create([parameters]) : BLOB 검출기 생성자
- cv2.SimpleBlobDetector_Params()
    - minThreshold, maxThreshold, thresholdStep : BLOB를 생성하기 위한 경계 값 (minThreshold에서 maxThreshold를 넘지않을 때 까지 thresholdStep만큼 증가)
    - minRepeatability : BLOB에 참여하기 위한 연속된 경계 값의 개수
    - minDistBetweenBlobs : 두 BLOB를 하나의 BLOB로 간주한 거리
    - filterByArea : 면적 필터 옵션
    - minArea, maxArea : min~max범위의 면적만 BLOB로 검출
    - filterByCircularity : 원형 비율 필터 옵션
    - minCircularity, maxCircularity : min~max범위의 원형의 비율만 BLOB로 검출
    - filterByColor : 밝기를 이용한 필터 옵션
    - blobColor : 0 = black / 255 = white BLOB검출
    - filterByConvexity : 볼록 비율 필터 옵션
    - minConvexity, maxConvexity : min~max범위의 볼록 비율만 BLOB로 검출
    - filterByInertia : 관성 비율 필터 옵션
    - minInertiaRatio, maxInertiaRatio : min~max범위의 관성 비율만 BLOB로 검출

In [22]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SimpleBlobDetector_create
detector = cv2.SimpleBlobDetector_create()
# key pts
keypoints = detector.detect(gray)
# key pts 그리기
img = cv2.drawKeypoints(img, keypoints, None, (0, 0, 255), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('BLOB', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SimpleBlobDetector_parameter 생성
params = cv2.SimpleBlobDetector_Params()

# 경계값 조정
params.minThreshold = 10
params.maxThreshold = 240
params.thresholdStep = 5

# 면적 필터를 켜고 최소 값 지정
params.filterByArea = True
params.minArea = 200

# 컬러, 볼록 비율, 원형 비율 필터 옵션 끄기
params.filterByColor = False
params.filterByConvexity = False
params.filterByInertia = False
params.filterByCircularity = False

# SimpleBlobDetector_create
detector = cv2.SimpleBlobDetector_create(params)
# key pts
keypoints = detector.detect(gray)
# key pts 그리기
img = cv2.drawKeypoints(img, keypoints, None, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('BLOB with Params', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 8.3 디스크립터 추출기
### 8.3.1 특징 디스크립터와 추출기
- 특징 디스크립터를 추출하기 위한 함수
- keypoints, descriptors = detector.compute(img, keypoints[, descriptors]) : 키포인트를 전달하면 특징 디스크립터를 계산해서 반환
- keypoints, descriptors = detector.detectAndCompute(img, mask[, descriptors, useProvideKeypoints]) : 키포인트 검출과 특징 디스크립터 계산을 한 번에 수행
    - keypoints : 디스크립터 계산을 위해 사용할 키포인트
    - descriptors : 계산된 디스크립터
    - mask : 키포인트 검출에 사용할 마스크
    - useProvideKeypoints : True 인 경우 키포인트 검출을 수행하지 않음 (사용 x)

### 8.3.2 SIFT (scale-invariant feature transform)
- 이미지 피라미드를 이용해 크기 변화에 따른 특징 검출의 문제를 해결한 알고리즘
- detector = cv2.xfeatures2d.SIFT_create([, nfeatures[, nOctavelLayers[, contrastThreshold[, edgeThreshold[, sigma]]]]])
    - nfeatures : 검출 최대 특징 수
    - nOctavelLayers : 이미지 피라미드에 사용할 계층 수
    - contrastThreshold : 필터링할 빈약한 특징 문턱 값
    - edgeThreshold : 필터링할 엣지 문턱 값
    - sigma : 이미지 피라미드 0계층에서 사용할 가우시안 필터의 시그마 값

In [27]:
import cv2
import numpy as np

img = cv2.imread('img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SIFT 생성
sift = cv2.xfeatures2d.SIFT_create()

# key pts
keypoints, descriptor = sift.detectAndCompute(gray, None)
print('keypoints: ', len(keypoints), 'descriptor: ', descriptor.shape)
print(descriptor)

# key pts 그리기
img = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('SIFT', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

keypoints:  413 descriptor:  (413, 128)
[[  1.   1.   1. ...   0.   0.   1.]
 [  8.  24.   0. ...   1.   0.   4.]
 [  0.   0.   0. ...   0.   0.   2.]
 ...
 [  1.   8.  71. ...  73. 127.   3.]
 [ 35.   2.   7. ...   0.   0.   9.]
 [ 36.  34.   3. ...   0.   0.   1.]]


## 8.4 특징 매칭 
- 서로 다른 두 영상에서 구한 키포인트와 특징 디스크립터들을 각각 비교해 그 거리가 비슷한 것끼리 짝짓는 것을 의미

### 8.4.1 특징 매칭 인터페이스

### 8.4.2 BFMatcher

In [30]:
import cv2
import numpy as np

img1 = cv2.imread('img/taekwonv1.jpg')
img2 = cv2.imread('img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# sift 디스크립터 추출기 생성
detector = cv2.xfeatures2d.SIFT_create()

# 각 영상에 대해 키포인트와 디스크립터 추출
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

# BFMatcher 생성, L1거리, 상호체크
matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
# 매칭 계산
matches = matcher.match(desc1, desc2)
# 매칭 결과 그리기
res = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# results
cv2.imshow('BFMater + SIFT', res)
cv2.waitKey()
cv2.destroyAllWindows()

In [43]:
### 8.4.4 좋은 매칭점 찾기
import cv2
import numpy as np

img1 = cv2.imread('img/taekwonv1.jpg')
img2 = cv2.imread('img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB 디스크립터 추출기 생성
detector = cv2.ORB_create()

# 각 영상에 대해 키포인트와 디스크립터 추출
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

# BF-Hamming으로 매칭
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(desc1, desc2)

# 매칭 결과 거리기준 오름차순 정렬
matches = sorted(matches, key=lambda x:x.distance)
# 최소 거리값과 최대 거리값 확보
min_dist, max_dist = matches[0].distance, matches[-1].distance
# 최소 거리의 20% 지점을 임계점으로 설정
ratio = 0.2
good_thresh = (max_dist - min_dist)*ratio + min_dist

# 임계점보다 작은 매칭점만 좋은 매칭점으로 분류
good_matches = [m for m in matches if m.distance < good_thresh]
print(f'matches : {len(good_matches)}/{len(matches)}, min : {min_dist:.2f}, max : {max_dist:.2f}, thresh: {good_thresh:.2f}')

# 좋은 매칭 결과 그리기
res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# results
cv2.imshow('Good Matches', res)
cv2.waitKey()
cv2.destroyAllWindows()

matches : 18/127, min : 24.00, max : 78.00, thresh: 34.80


In [10]:
import cv2, numpy as np

img1 = None
win_name = 'Camera Matching'
MIN_MATCH = 10
# ORB 검출기 생성  ---①
detector = cv2.ORB_create(1000)
# Flann 추출기 생성 ---②
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6,
                   key_size = 12,
                   multi_probe_level = 1)
search_params=dict(checks=32)
matcher = cv2.FlannBasedMatcher(index_params, search_params)
# 카메라 캡쳐 연결 및 프레임 크기 축소 ---③
cap = cv2.VideoCapture(0)              
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cap.isOpened():       
    ret, frame = cap.read() 
    if img1 is None:  # 등록된 이미지 없음, 카메라 바이패스
        res = frame
    else:             # 등록된 이미지 있는 경우, 매칭 시작
        img2 = frame
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        # 키포인트와 디스크립터 추출
        kp1, desc1 = detector.detectAndCompute(gray1, None)
        kp2, desc2 = detector.detectAndCompute(gray2, None)
        # k=2로 knnMatch
        matches = matcher.knnMatch(desc1, desc2, 2)
        # 이웃 거리의 75%로 좋은 매칭점 추출---②
        ratio = 0.75
        good_matches = [m[0] for m in matches \
                if len(m) == 2 and m[0].distance < m[1].distance * ratio]
        print('good matches:%d/%d' %(len(good_matches),len(matches)))
        # 모든 매칭점 그리지 못하게 마스크를 0으로 채움
        matchesMask = np.zeros(len(good_matches)).tolist()
        # 좋은 매칭점 최소 갯수 이상 인 경우
        if len(good_matches) > MIN_MATCH: 
            # 좋은 매칭점으로 원본과 대상 영상의 좌표 구하기 ---③
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ])
            # 원근 변환 행렬 구하기 ---⑤
            mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            accuracy=float(mask.sum()) / mask.size
            print("accuracy: %d/%d(%.2f%%)"% (mask.sum(), mask.size, accuracy))
            if mask.sum() > MIN_MATCH:  # 정상치 매칭점 최소 갯수 이상 인 경우
                # 이상점 매칭점만 그리게 마스크 설정
                matchesMask = mask.ravel().tolist()
                # 원본 영상 좌표로 원근 변환 후 영역 표시  ---⑦
                h,w, = img1.shape[:2]
                pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ])
                dst = cv2.perspectiveTransform(pts,mtrx)
                img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        # 마스크로 매칭점 그리기 ---⑨
        res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, \
                            matchesMask=matchesMask,
                            flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    # 결과 출력
    cv2.imshow(win_name, res)
    key = cv2.waitKey(1)
    if key == 27:    # Esc, 종료
            break          
    elif key == ord(' '): # 스페이스바를 누르면 ROI로 img1 설정
        x,y,w,h = cv2.selectROI(win_name, frame, False)
        if w and h:
            img1 = frame[y:y+h, x:x+w]
else:
    print("can't open camera.")
cap.release()                          
cv2.destroyAllWindows()

good matches:18/399
accuracy: 9/18(0.50%)
good matches:21/399
accuracy: 13/21(0.62%)
good matches:23/399
accuracy: 14/23(0.61%)
good matches:17/399
accuracy: 12/17(0.71%)
good matches:17/399
accuracy: 9/17(0.53%)
good matches:12/399
accuracy: 8/12(0.67%)
good matches:19/399
accuracy: 10/19(0.53%)
good matches:22/399
accuracy: 13/22(0.59%)
good matches:15/399
accuracy: 10/15(0.67%)
good matches:17/399
accuracy: 9/17(0.53%)
good matches:16/399
accuracy: 10/16(0.62%)
good matches:25/399
accuracy: 14/25(0.56%)
good matches:20/399
accuracy: 10/20(0.50%)
good matches:28/399
accuracy: 12/28(0.43%)
good matches:24/399
accuracy: 13/24(0.54%)
good matches:19/399
accuracy: 11/19(0.58%)
good matches:14/399
accuracy: 7/14(0.50%)
good matches:15/399
accuracy: 11/15(0.73%)
good matches:17/399
accuracy: 10/17(0.59%)
good matches:20/399
accuracy: 12/20(0.60%)
good matches:24/399
accuracy: 14/24(0.58%)
good matches:21/399
accuracy: 12/21(0.57%)
good matches:17/399
accuracy: 10/17(0.59%)
good matches:25/

In [11]:
## 8.5 객체 추적
### 8.5.1 동영상 배경 제거
### MOG2 배경 제거
import numpy as np
import cv2

cap = cv2.VideoCapture('img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

# 배경 제거 객체 생성
fgbg = cv2.createBackgroundSubtractorMOG2()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 배경 제거 마스크 계산
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame', frame)
    cv2.imshow('bgsub', fgmask)
    if cv2.waitKey(delay) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [8]:
### p.358 희소 옵티컬 플로
import numpy as np
import cv2

cap = cv2.VideoCapture('img/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
delay = int(1000/fps)
# 추적 경로를 그리기 위한 랜덤 색상
color = np.random.randint(0,255,(200,3))
lines = None  #추적 선을 그릴 이미지 저장 변수
prevImg = None  # 이전 프레임 저장 변수
# calcOpticalFlowPyrLK 중지 요건 설정
termcriteria =  (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        break
    img_draw = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 최초 프레임 경우
    if prevImg is None:
        prevImg = gray
        # 추적선 그릴 이미지를 프레임 크기에 맞게 생성
        lines = np.zeros_like(frame)
        # 추적 시작을 위한 코너 검출  ---①
        prevPt = cv2.goodFeaturesToTrack(prevImg, 200, 0.01, 10)
    else:
        nextImg = gray
        # 옵티컬 플로우로 다음 프레임의 코너점  찾기 ---②
        nextPt, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, \
                                        prevPt, None, criteria=termcriteria)
        # 대응점이 있는 코너, 움직인 코너 선별 ---③
        prevMv = prevPt[status==1]
        nextMv = nextPt[status==1]
        for i,(p, n) in enumerate(zip(prevMv, nextMv)):
            px,py = p.ravel()
            nx,ny = n.ravel()
            # 이전 코너와 새로운 코너에 선그리기 ---④
            cv2.line(lines, (int(px), int(py)), (int(nx), int(ny)), color[i].tolist(), 2)
            # 새로운 코너에 점 그리기
            cv2.circle(img_draw, (int(nx), int(ny)), 2, color[i].tolist(), -1)
        # 누적된 추적 선을 출력 이미지에 합성 ---⑤
        img_draw = cv2.add(img_draw, lines)
        # 다음 프레임을 위한 프레임과 코너점 이월
        prevImg = nextImg
        prevPt = nextMv.reshape(-1,1,2)

    cv2.imshow('OpticalFlow-LK', img_draw)
    key = cv2.waitKey(delay)
    if key == 27 : # Esc:종료
        break
    elif key == 8: # Backspace:추적 이력 지우기
        prevImg = None
cv2.destroyAllWindows()
cap.release()

In [11]:
### CamShift 객체 추적
### p.358 희소 옵티컬 플로
import numpy as np
import cv2

roi_hist = None # 추적 객체 히스토그램 저장 변수
win_name = 'Camshift Tracking'
termination =  (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


while cap.isOpened():
    ret,frame = cap.read()
    img_draw = frame.copy()
    
    if roi_hist is not None:
        # 전체 영상 hsv 컬러로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # 전체 영상 히스토그램과 roi 히스토그램 역투영
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # 역투영 결과와 초기 추적 위치로 평균 이동 추적
        ret, (x, y, w, h) = cv2.CamShift(dst, (x, y, w, h), termination)
        # 새로운 위치에 사각형 표시
        cv2.rectangle(img_draw, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # 컬러 영상과 역투영 영상을 통합해 출력
        result = np.hstack((img_draw, cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)))
    
    else:
        cv2.putText(img_draw, 'Hit the spacce to set target to track', (10, 30),
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
        result = img_draw

    cv2.imshow(win_name, result)
    key = cv2.waitKey(1) & 0xFF
    if key == 27 : # Esc:종료
        break
    elif key == ord(' '): # 스페이스바, ROI 설정
        x, y, w, h = cv2.selectROI(win_name, frame, False)
        if w and h:
            roi = frame[y:y+h, x:x+w]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = None
            roi_hist = cv2.calcHist([roi], [0], mask, [180], [0, 180])
            cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
    else:
        roi_hist = None
else:
    print('No camera!')

cap.release()
cv2.destroyAllWindows()